### Prepare Imaging for Source Localization

In [ ]:
fwd_name=f'{output_dir}/{subjid}-fwd.fif'

In [ ]:
#Requires Freesurfer -  This is already done for our subject
#mne.bem.make_watershed_bem(subjid, subjects_dir=subjects_dir)

In [ ]:
import nih2mne
from nih2mne import calc_mnetrans

In [ ]:
trans_fname = f'{topdir}/{subjid}-trans.fif'
if not os.path.exists(trans_fname):
    # The calc_mnetrans.py code takes tagged Afni, Brainsight, or BIDS json
    anat_json = f'{topdir}/{subjid}/ses-01/meg/{subjid}_ses-01_task-airpuff_run-01_coordsystem.json'
    ! calc_mnetrans.py -anat_json {anat_json} -trans_output {subjid}-trans.fif -dsname {meg_fname} -subject {subjid} -subjects_dir {subjects_dir}
    trans = mne.read_trans(trans_fname)
else:
    trans = mne.read_trans(trans_fname)

### Plot the transform

In [ ]:
mne.viz.plot_alignment(raw.info, trans=trans, subject=subjid, subjects_dir=subjects_dir, dig=True)

In [ ]:
fwd_name=f'{output_dir}/{subjid}_vol-fwd.fif'
if not os.path.exists(fwd_name):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    bem = mne.make_bem_model(subjid, subjects_dir=subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    src = mne.source_space.setup_volume_source_space(subject=subjid, subjects_dir=subjects_dir,
                                                     mri='T1.mgz', bem=bem_sol) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(fwd_name, forward)
else:
    forward = mne.read_forward_solution(fwd_name)

## Volumetric Source Model 

In [ ]:
src = forward['src']
src.plot(subjid, subjects_dir=subjects_dir, brain=False, trans=trans)

### Surface Source

In [ ]:
fwd_name=f'{output_dir}/{subjid}-fwd.fif'
if not os.path.exists(fwd_name):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    bem = mne.make_bem_model(subjid, subjects_dir=subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    src = mne.source_space.setup_source_space(subject=subjid, subjects_dir=subjects_dir, n_jobs=-1) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(fwd_name, forward)
else:
    forward = mne.read_forward_solution(fwd_name)

In [ ]:
src = forward['src']
src.plot(subjects_dir=subjects_dir, head=False, brain='white', trans=trans)

## Boundary Element Model (For MEG - red boundary in figure is all that is necessary)

In [ ]:
%matplotlib inline
_ = mne.viz.plot_bem(subject=subjid, subjects_dir=subjects_dir)

## Spatial relationship of the brain to the MEG sensors

In [ ]:
mne.viz.plot_alignment(raw.info, trans=trans, subject=subjid, subjects_dir=subjects_dir, dig=True, surfaces='pial')

## What is a forward Model?
## Simulation of all the source locations through the BEM

In [ ]:
print(forward['sol']['data'].shape)
print('Channels, Source Locations X 3')

In [ ]:
from mne.viz.topomap import plot_topomap

In [ ]:
tmp=raw.copy()
grads = [i for i in tmp.ch_names if i[0]=='M']
_=tmp.pick_channels(grads)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(3,2)
axs[0,0].set_title('Location 1 Forward Model')
plot_topomap(forward['sol']['data'][:,1003], tmp.info, axes=axs[0,0])
plot_topomap(forward['sol']['data'][:,1004], tmp.info, axes=axs[1,0])
plot_topomap(forward['sol']['data'][:,1005], tmp.info, axes=axs[2,0])

axs[0,1].set_title('Location 2 Forward Model')
plot_topomap(forward['sol']['data'][:,5003], tmp.info, axes=axs[0,1])
plot_topomap(forward['sol']['data'][:,5004], tmp.info, axes=axs[1,1])
plot_topomap(forward['sol']['data'][:,5005], tmp.info, axes=axs[2,1])


In [ ]:
%matplotlib inline
fig=pylab.figure(figsize=(12,8))
pylab.imshow(forward['sol']['data'], aspect='auto', cmap='jet', vmax=3e-6, vmin=-3e-6)
pylab.title('Forward Model Simulation at every point in the source model')
pylab.xlabel('Source Location')
pylab.ylabel('Channel index')